In [ ]:
!pip install huggingface_hub
!pip install cohere
!pip install transformers
!pip install sumy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 23.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 61.6 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21691 sha256=36f779b79c2f67cc7765aa2ee548482a938db4337c18a2e5584e713affb51f08
  Stored in directory: /root/.cache/pip/wheels/4d/57/58/7e3d7fedf51fe248b7fcee3df6945ae28638e22cddf01eb92b
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=795c09c93484c9dd25b2ae80af1c08a5b7f903bc0189ce547cc7540e9795b5b4
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built breadability docopt


# Loading Hugging Face Data

In [ ]:
from huggingface_hub import hf_hub_download
import pandas as pd

# huggingface repo id
repo_id = "abisee/cnn_dailymail"


filenames = [
    "3.0.0/test-00000-of-00001.parquet",
    "3.0.0/train-00000-of-00003.parquet",
    "3.0.0/train-00001-of-00003.parquet",
    "3.0.0/train-00002-of-00003.parquet",
    "3.0.0/validation-00000-of-00001.parquet"
]

dfs = {
    filename: pd.read_parquet(hf_hub_download(
        repo_id=repo_id, filename=filename, repo_type="dataset"
    ))
    for filename in filenames
}


test_df = dfs["3.0.0/test-00000-of-00001.parquet"]
train_df = pd.concat([dfs[f] for f in filenames if "train" in f], ignore_index=True)
val_df = dfs["3.0.0/validation-00000-of-00001.parquet"]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

##Storing to S3

In [ ]:
#save file to parquet

train_df.to_parquet('train.parquet')


In [ ]:
import boto3
def upload_to_s3(file_path, bucket_name, s3_key):
  s3 = boto3.client('s3')
  try:
    s3.upload_file(file_path, bucket_name, s3_key)
    print(f"File '{file_path}' successfully uploaded to 's3://{bucket_name}/{s3_key}'")
  except Exception as e:
    print(f"Error uploading file: {e}")

ModuleNotFoundError: No module named 'boto3'

##Cohere API

In [ ]:
import cohere

co = cohere.Client('OqqyzITiJdIuS4hVpIJT7V70qcknzNGn2EAbias1')

def generate_embedding(content):
    if content:
        response = co.embed(texts=[content], model='embed-english-v2.0')
        return response.embeddings[0]
    return None

def cohere_prompt(prompt, model='command', max_tokens=300):
    response = co.generate(
        model=model,
        prompt=prompt,
        max_tokens=max_tokens
    )
    return response.generations[0].text.strip()

## Abstractive Summary Generation

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-small')
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-small')

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def summarize_text(content):
    if content:
        input_text = f"summarize: {content}"  # T5 expects a 'summarize:' prefix
        inputs = tokenizer(input_text, return_tensors='pt', max_length=512, truncation=True)
        summary_ids = model.generate(inputs.input_ids, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
        return tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return None

## Extractive Summary Generation Function

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

def extractive_summary(text, sentence_count=4):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentence_count)

    return " ".join(str(sentence) for sentence in summary)

https://www.geeksforgeeks.org/mastering-text-summarization-with-sumy-a-python-library-overview/

##Similarity Search

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def get_top_k_similar_articles(query, df, k=3):
    query_embedding = np.array(generate_embedding(query)).reshape((1, -1))
    df_filtered = df[df['Embedding'].notnull()].copy()
    embeddings = np.stack(df_filtered['Embedding'].values)
    similarities = cosine_similarity(query_embedding, embeddings).flatten()
    df_filtered['Similarity'] = similarities
    top_k = df_filtered.sort_values('Similarity', ascending=False).head(k)
    return top_k

##Final Summary

In [ ]:
def generate_summaries(query, df, k=5):
    # Get top-k articles
    top_k_articles = get_top_k_similar_articles(query, df, k)

    # Extractive summaries
    extractive_summaries = "ARTICLE: " + "\n\nARTICLE: ".join(top_k_articles['Extractive Summary'].values)
    extractive_prompt = f"Summarize the following articles into a cohesive extractive summary in under 200 words with proper flow. It should have something from each article:\n{extractive_summaries}"

    # Abstractive summaries
    abstractive_summaries = "ARTICLE: " + "\n\nARTICLE: ".join(top_k_articles['Abstractive Summary'].values)
    abstractive_prompt = f"Summarize the following articles into a cohesive abstractive summary in under 200 words with proper flow. It should have something from each article:\n{abstractive_summaries}"

    print("EXTRACTIVE PROMPT:\n", extractive_prompt)
    extractive_summary = cohere_prompt(extractive_prompt)

    print("\n\nABSTRACTIVE PROMPT:\n", abstractive_prompt)
    abstractive_summary = cohere_prompt(abstractive_prompt)

    return extractive_summary, abstractive_summary